# Example of how to use

In [23]:
import enum
from langchain_core.messages.base import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import logging as log
import os
import pprint

In [14]:
log.basicConfig(level=log.INFO, format='%(asctime)s [%(levelname)5s] %(message)s',datefmt='%H:%M:%S')

In [24]:
class FinishReason(enum.Enum):
    Stop=0,
    LengthExceeded=1

def parseFinishReason( message: BaseMessage):
    finishReason= message.response_metadata['finish_reason']
    if 'STOP'==finishReason:
        return FinishReason.Stop        
    elif 'LENGTH'==finishReason:       
        return FinishReason.LengthExceeded
    raise ValueError(f'Invalid finish reason:{s}')

In [15]:
API_KEY='GOOGLE_API_KEY'
MODEL_NAME='gemini-1.5-flash'
assert API_KEY in os.environ 
assert len( str(API_KEY).strip() ) > 0
log.info(f'Got envvar:{API_KEY}.')

16:14:51 [ INFO] - Got envvar:GOOGLE_API_KEY.


In [16]:
model = ChatGoogleGenerativeAI(model=MODEL_NAME, 
                               max_output_tokens=100,
                               temperature=0.0)

template= ChatPromptTemplate([('system','Translate the following text into {language}.'),
                             ('human','{text}')])
parser= StrOutputParser()

chain = template | model | parser
result= chain.invoke({ 'language':'Italian', 'text':'Hi, how are you doing today?'})
result

'Ciao, come stai oggi?'

In [17]:
pprint.pprint(result)

'Ciao, come stai oggi?'


In [26]:
template= ChatPromptTemplate([('system','Translate the following text into {language}.'),
                             ('human','{text}')])
parser= StrOutputParser()

chainToResult = template | model
result= chainToResult.invoke({ 'language':'Italian', 'text':'Hi, how are you doing today?'})
result.pretty_print()

finishReason= parseFinishReason(result)
print(result.response_metadata)
print(f'Finish reason: {finishReason.name}')

================================== Ai Message ==================================

Ciao, come stai oggi?
{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}
Finish reason: Stop


In [20]:
strResult= parser.invoke(result)
strResult

'Ciao, come stai oggi?'